In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow import keras
import pandas as pd
import gc
import numpy as np
import cv2

import rasterio
from rasterio.windows import Window
import pathlib
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

In [ ]:
# https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# thanks to @igor14497 for helping me debug
def make_grid(shape, window=512):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape

    if x % WINDOW == 0:
        nx = x // window
    else:
        nx = x // window + 1

    x1 = list(range(0, x, window))
    x2 = [(x + window) for x in x1][:-1]
    x2.append(x)

    if y % WINDOW == 0:
        ny = y // window
    else:
        ny = y // window + 1

    y1 = list(range(0, y, window))
    y2 = [(y + window) for y in y1][:-1]
    y2.append(y)

    slices = np.zeros((nx, ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny, 4)

In [ ]:
THRESHOLD = 0.4
WINDOW = 4*512
TILE_SIZE = 512

In [ ]:
MODEL = '../input/notebook4ca5c6eb1c/alina_resnet34.h5'
model = keras.models.load_model(MODEL, compile=False)
print(model)

In [ ]:
# https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-subm

sub = {}

p = pathlib.Path('../input/hubmap-kidney-segmentation')
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):
    print(f'{i+1} Predicting {filename.stem}')
    
    dataset = rasterio.open(filename.as_posix(), transform=identity)
    preds = np.zeros(dataset.shape, dtype=np.uint8)    

    slices = make_grid(dataset.shape, window=WINDOW)

    for (x1,x2,y1,y2) in slices:
        if (x2-x1) == WINDOW and (y2-y1) == WINDOW:
            image = dataset.read([1,2,3],
                                 window=Window.from_slices((x1,x2),(y1,y2)))
            image = np.moveaxis(image, 0, -1)
            image = cv2.resize(image, (TILE_SIZE, TILE_SIZE), interpolation=cv2.INTER_AREA)
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image = np.expand_dims(image, 0)

            pred =  np.squeeze(model.predict(image))
            pred = cv2.resize(pred, (WINDOW, WINDOW), interpolation=cv2.INTER_NEAREST)
            preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)
        else:
            continue

    # preds = (preds > 0.5).astype(np.uint8)
    
    sub[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
    
    del preds
    gc.collect();

In [ ]:
submission = pd.DataFrame.from_dict(sub, orient='index')
submission.to_csv('submission.csv', index=False)
submission.head()